In [1]:
import numpy as np
import pandas as pd
import os
import tifffile
import torch 
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import rasterio
from rasterio.plot import show
from PIL import Image
from utils.data_handlers import *
from utils.models import *
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
# Example usage
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
import copy
import random

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
train_labels_1, train_artifacts_1= link_label1_to_image('train/image', 'train/label2')


In [4]:
all_labels_0, all_artifacts_0=link_label0_to_image('No_Artifacts','No_Artifacts_label')



In [5]:
keys = list(all_labels_0.keys())


# Split the keys into training and testing sets
keys_train, keys_test = train_test_split(keys, test_size=0.2, random_state=42)
train_labels_0 = {key: all_labels_0[key] for key in keys_train}
train_artifacts_0 = {key: all_artifacts_0[key] for key in keys_train}
test_labels_0 = {key: all_labels_0[key] for key in keys_test}
test_artifacts_0 = {key: all_artifacts_0[key] for key in keys_test}


In [6]:
train_labels_1.update(train_labels_0)
train_labels=train_labels_1
train_artifacts_1.update(train_artifacts_0)
train_artifacts=train_artifacts_1


In [7]:
print(len(train_labels),len(train_artifacts))

347 347


In [8]:
test_labels_1, test_artifacts_1=link_label1_to_image('test/image', 'test/label2')
print(len(test_labels_1), len(test_artifacts_1))

296 296


In [9]:
test_labels_1.update(test_labels_0)
test_labels=test_labels_1
test_artifacts_1.update(test_artifacts_0)
test_artifacts=test_artifacts_1
print(len(test_labels),len(test_artifacts))

347 347


In [10]:
test_labels.update(train_labels)
total_labels=test_labels
test_artifacts.update(train_artifacts)
total_artifacts=test_artifacts
print(len(test_labels),len(test_artifacts))

694 694


In [11]:
keys = list(total_labels.keys())


keys_train, keys_temp = train_test_split(keys, test_size=0.3, random_state=42)
keys_val, keys_test = train_test_split(keys_temp, test_size=0.5, random_state=42)

# Create dictionaries for training, validation, and testing data
train_labels_final = {key: total_labels[key] for key in keys_train}
train_artifacts_final = {key: total_artifacts[key] for key in keys_train}
val_labels = {key: total_labels[key] for key in keys_val}
val_artifacts = {key: total_artifacts[key] for key in keys_val}
test_labels = {key: total_labels[key] for key in keys_test}
test_artifacts = {key: total_artifacts[key] for key in keys_test}

In [12]:
output_folder = 'train'
train_dict=save_cropped_images_and_labels(train_artifacts_final, output_folder)

In [13]:
len(train_dict)

4365

In [14]:
output_folder = 'val'
val_dict=save_cropped_images_and_labels(val_artifacts, output_folder)

In [15]:
len(val_dict)

936

In [16]:
output_folder = 'test'
test_dict=save_cropped_images_and_labels(test_artifacts, output_folder)

In [17]:
len(test_dict)

945

In [18]:


train_data = CustomDataset(train_dict)
test_data = CustomDataset(test_dict)
val_data = CustomDataset(val_dict)


In [19]:
train_keys = list(train_dict.keys())
val_keys = list(val_dict.keys())
test_keys = list(test_dict.keys())

# Sample subsets from the dictionaries
subset_train_dict = {k: train_dict[k] for k in random.sample(train_keys, min(100, len(train_keys)))}
subset_val_dict = {k: val_dict[k] for k in random.sample(val_keys, min(25, len(val_keys)))}
subset_test_dict = {k: test_dict[k] for k in random.sample(test_keys, min(100, len(test_keys)))}

In [20]:
subset_train_data = CustomDataset(subset_train_dict)
subset_val_data = CustomDataset(subset_val_dict)
subset_test_data = CustomDataset(subset_test_dict)

In [21]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=True)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)



In [22]:
subset_train_loader = DataLoader(subset_train_data, batch_size=64, shuffle=True)
subset_val_loader = DataLoader(subset_val_data, batch_size=16, shuffle=True)
subset_test_loader = DataLoader(subset_test_data, batch_size=len(subset_test_data), shuffle=True)

In [23]:
model = UNet(pretrained=True)  # Assuming input grayscale images and 2 classes (background and artifacts)
criterion = dice_loss
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Training
model.train_model(model, subset_train_loader, subset_val_loader, criterion, optimizer,num_epochs=1,model_save_path='best_train_model.pth')



C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/1, Loss: 0.13839031755924225
Epoch 1/1, Train Loss: 0.13839031755924225, Val Loss: 0.1378256231546402
Training completed.


In [24]:
model.test_model(model=model, test_loader=test_loader, criterion=criterion, output_folder='test_output')

Test Loss:0.13820317387580872
